In [2]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

import warnings
warnings.filterwarnings('ignore')

In [3]:
_  = load_dotenv(find_dotenv())

openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_version = "2023-07-01-preview"
openai.api_type = "azure"

In [4]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [5]:
def get_completion(prompt, model=llm_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
        engine='test',
    )
    return response.choices[0].message["content"]

In [6]:
get_completion('Hello, how are you?')

"Hello! I'm an AI language model, so I don't have feelings, but I'm here to help you. How can I assist you today?"

## Langchain: Memory

### 1. Conversation Buffer Memory

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [8]:
llm = ChatOpenAI(temperature=0, model=llm_model, engine='test')
memory = ConversationBufferMemory()
conversation = ConversationChain(llm=llm, memory=memory, verbose=True)

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [9]:
conversation.predict(input='Hi, my name is John. What is your name?')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is John. What is your name?
AI:

> Finished chain.


"Hello John! I'm an AI language model developed by OpenAI, and I don't have a specific name. You can just call me AI. How can I assist you today?"

In [10]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is John. What is your name?
AI: Hello John! I'm an AI language model developed by OpenAI, and I don't have a specific name. You can just call me AI. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'1+1 equals 2.'

In [11]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is John. What is your name?
AI: Hello John! I'm an AI language model developed by OpenAI, and I don't have a specific name. You can just call me AI. How can I assist you today?
Human: What is 1+1?
AI: 1+1 equals 2.
Human: What is my name?
AI:

> Finished chain.


'Your name is John.'

In [12]:
print(memory.buffer)

Human: Hi, my name is John. What is your name?
AI: Hello John! I'm an AI language model developed by OpenAI, and I don't have a specific name. You can just call me AI. How can I assist you today?
Human: What is 1+1?
AI: 1+1 equals 2.
Human: What is my name?
AI: Your name is John.


In [13]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is John. What is your name?\nAI: Hello John! I'm an AI language model developed by OpenAI, and I don't have a specific name. You can just call me AI. How can I assist you today?\nHuman: What is 1+1?\nAI: 1+1 equals 2.\nHuman: What is my name?\nAI: Your name is John."}

In [14]:
memory = ConversationBufferMemory()

In [15]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [16]:
print(memory.buffer)

Human: Hi
AI: What's up


In [17]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [18]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [19]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

### ConversationBufferWindowMemory

In [20]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)

parameter k adalah jumlah kata yang diambil dari window sebelum dan sesudah kata yang sedang diproses

In [21]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})


In [22]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [23]:
llm = ChatOpenAI(temperature=0.0, model=llm_model, engine='test')
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [24]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:



> Finished chain.


"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [25]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'1+1 is equal to 2.'

In [26]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my name?
AI:

> Finished chain.


"I'm sorry, but I don't have access to personal information about individuals unless it has been shared with me in the course of our conversation."

### 3. Conversation Token Buffer Memory

In [27]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI

In [30]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})